# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet("./goodsread/items.par")
events = pd.read_parquet("./goodsread/events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = list(set(users_train) & set(users_test))

print(len(users_train), len(users_test), len(common_users))

444517 123818 120858


In [6]:
cold_users = set(users_test) - set(common_users)

print(len(cold_users)) 

2960


In [10]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(by="popularity_weighted",ascending=False).reset_index()

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.loc[item_popularity["avg_rating"] >= 4][0:100]
top_k_pop_items

,index,item_id,users,avg_rating,popularity_weighted
2,32397,18007564,20231,4.321240,87423.0
3,32633,18143977,19500,4.290872,83672.0
4,30704,16096824,16793,4.301316,72232.0
5,2,3,15191,4.706076,71490.0
7,3718,38447,14651,4.233704,62028.0
...,...,...,...,...,...
131,35414,23093367,4330,4.454503,19288.0
132,378,3636,4677,4.098781,19170.0
134,32845,18293427,4677,4.092367,19140.0
135,33621,18966819,4366,4.375401,19103.0


In [11]:
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_weighted,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20231,4.321240,87423.0,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19500,4.290872,83672.0,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16793,4.301316,72232.0,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15191,4.706076,71490.0,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
4,38447,Margaret Atwood,The Handmaid's Tale,1998,14651,4.233704,62028.0,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
5,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13051,4.632442,60458.0,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14364,4.179476,60034.0,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12190,4.730353,57663.0,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,18692431,"Nicola Yoon, David Yoon","Everything, Everything",2015,14141,4.071706,57578.0,"{'Young Adult': 5175, 'Romance': 3234, 'Contem..."
9,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11900,4.770000,56763.0,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."


# === Знакомство: первые персональные рекомендации

In [14]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items, on="item_id", how="left")
cold_users_events_with_recs

,item_id,started_at,read_at,is_read,rating,is_reviewed,user_id,index,users,avg_rating,popularity_weighted,author,title,genre_and_votes,publication_year
0,6900,2017-10-09,2017-10-13,True,4,False,1375894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1,12555,2017-09-21,2017-10-11,True,3,False,1375894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
2,25899336,2017-09-12,2017-09-17,True,4,True,1375894,36924.0,4810.0,4.428274,21300.0,Paul Kalanithi,When Breath Becomes Air,"{'Nonfiction': 5848, 'Autobiography-Memoir': 3...",<NA>
3,21936809,2017-08-20,2017-08-24,True,4,True,1375894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4,6952,2017-09-18,2017-09-20,True,3,False,1375894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10262,16181775,2017-09-24,2017-09-25,True,3,False,1263231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
10263,10210,2017-09-16,2017-09-24,True,5,False,1263231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
10264,7907782,2017-08-15,2017-09-05,True,4,False,1263231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
10265,317375,2017-08-06,2017-08-10,True,5,False,1335389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [15]:

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]] 

In [16]:
cold_user_recs

,user_id,item_id,rating,avg_rating
2,1375894,25899336,4,4.428274
5,1352463,16096824,5,4.301316
8,1352463,18692431,5,4.071706
9,1352463,28763485,2,4.195020
16,1286988,38447,5,4.233704
...,...,...,...,...
10242,1403739,22055262,4,4.138950
10249,1002519,17675462,5,4.097095
10252,1106396,10964,5,4.126443
10257,1203738,9969571,5,4.289925


In [17]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2)) 

0.77 0.62


In [18]:
cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}") 

Доля пользователей без релевантных рекомендаций: 0.64
Среднее покрытие пользователей: 0.49


# === Базовые подходы: коллаборативная фильтрация

In [19]:
events[['user_id', 'item_id', 'rating']]

,user_id,item_id,rating
0,1238283,22034,5
1,1238283,22318578,5
2,1238283,22551730,4
3,1238283,22816087,5
5,1238283,17910054,3
...,...,...,...
12914452,1378860,5297,5
12914453,1378860,4900,2
12914454,1378860,14836,3
12914456,1308836,10210,5


In [20]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set) 

In [21]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set) 

In [23]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8289
MAE:  0.6476
0.8288686648707771 0.6475568851342635


In [24]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set) 
mae = accuracy.mae(random_predictions)
print(mae)

MAE:  0.9989
0.9988512459358723


In [34]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events.query("user_id == @user_id")['item_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"]) 

In [36]:
rec_list = get_recommendations_svd(1296647, items, events_train, svd_model) 
rec_list

,item_id,score
0,1,5
1,2,5
2,3,5
3,5,5
4,6,5


In [37]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

user_id: 1378002
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
56,Douglas Hulick,"Among Thieves (Tales of the Kin, #1)",2013-09-27,2013-10-07,4,"{'Fantasy': 1170, 'Fiction': 73, 'Fantasy-Epic..."
57,H.G. Wells,The Island of Dr. Moreau,2013-09-21,2013-09-27,3,"{'Classics': 3115, 'Science Fiction': 2365, 'F..."
58,Hugh Howey,"Dust (Silo, #3)",2013-09-16,2013-09-22,4,"{'Science Fiction': 1086, 'Fiction': 598, 'Sci..."
59,Hugh Howey,"Shift (Shift, #1-3; Silo, #2)",2013-09-05,2013-09-16,3,"{'Science Fiction': 1226, 'Fiction': 652, 'Sci..."
60,Hugh Howey,"Wool Omnibus (Silo, #1)",2013-08-27,2013-09-05,4,"{'Science Fiction': 3107, 'Fiction': 1607, 'Sc..."
61,"Ryū Murakami, Ralph McCarthy",Audition,2013-07-29,2013-07-31,3,"{'Horror': 366, 'Fiction': 196, 'Asian Literat..."
62,Robert Ludlum,"The Bourne Identity (Jason Bourne, #1)",2013-07-16,2013-07-29,3,"{'Fiction': 2575, 'Thriller': 2075, 'Mystery':..."
63,John Scalzi,Redshirts,2013-07-12,2013-07-15,4,"{'Science Fiction': 3465, 'Fiction': 1113, 'Hu..."
64,John le Carré,The Spy Who Came In from the Cold,2013-07-09,2013-07-12,5,"{'Fiction': 1963, 'Thriller': 893, 'Mystery': ..."
65,Kristin Cashore,"Bitterblue (Graceling Realm, #3)",2013-05-17,2013-05-20,4,"{'Fantasy': 5470, 'Young Adult': 2592, 'Romanc..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,54741,4.794601,Quino,Toda Mafalda,"{'Sequential Art-Comics': 157, 'Humor': 47, 'S..."
1,11221285,4.766989,Brandon Sanderson,"The Way of Kings, Part 2 (The Stormlight Archi...","{'Fantasy': 641, 'Fiction': 46, 'Fantasy-Epic ..."
2,5031805,4.690915,"Anonymous, Lane T. Dennis, Wayne Grudem",ESV Study Bible,"{'Christian': 114, 'Religion-Theology': 100, '..."
3,862041,4.682133,J.K. Rowling,"Harry Potter Boxset (Harry Potter, #1-7)","{'Fantasy': 3211, 'Young Adult': 1068, 'Fictio..."
4,24812,4.675802,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."


# === Базовые подходы: контентные рекомендации

# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков